# Project description

This project corresponds to the development of a machine learning model for the prediction of credit scores, produced as the final assignment for the Machine Learning course at Nova SBE during the sprong semester of 2025.

Collaborators of this project are:
- Luis Silva (43055)
- Martin Hunger (65604)
- Mikel Irizar (63619)

# EDA

In [ ]:
import pandas as pd

# Had to "pip install openpyxl"

dataset = pd.read_excel("pr11_credit_scores.xlsx")


In [ ]:
dataset.info()

# The fact that we have some variables as objects, when they should be strings or numbers, points us to the fact that there are sveral data types mixed together (which should not happen) and alter us to data cleaning.

In [ ]:
dataset.describe(include="all")

In [54]:
pd.set_option('display.max_columns', None)
dataset.head(32)

,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3.0,4.0,3.0,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7,11.27,4.0,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3.0,4.0,3.0,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",-1,NaN,11.27,4.0,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3.0,4.0,3.0,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7,_,4.0,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3.0,4.0,3.0,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",5,4,6.27,4.0,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3.0,4.0,3.0,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",6,NaN,11.27,4.0,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good
5,0x1607,CUS_0xd40,June,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3.0,4.0,3.0,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",8,4,9.27,4.0,Good,809.98,27.262259,22 Years and 6 Months,No,49.574949,62.430172331195294,!@9#%8,340.4792117872438,Good
6,0x1608,CUS_0xd40,July,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3.0,4.0,3.0,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,8_,11.27,4.0,Good,809.98,22.537593,22 Years and 7 Months,No,49.574949,178.3440674122349,Low_spent_Small_value_payments,244.5653167062043,Good
7,0x1609,CUS_0xd40,August,NaN,23,#F%$D@*&8,Scientist,19114.12,1824.843333,3.0,4.0,3.0,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,6,11.27,4.0,Good,809.98,23.933795,NaN,No,49.574949,24.785216509052056,High_spent_Medium_value_payments,358.12416760938714,Standard
8,0x160e,CUS_0x21b1,January,Rick Rothackerj,28_,004-07-5839,_______,34847.84,3037.986667,2.0,4.0,6.0,1,Credit-Builder Loan,3,4,5.42,2.0,Good,605.03,24.464031,26 Years and 7 Months,No,18.816215,104.291825168246,Low_spent_Small_value_payments,470.69062692529184,Standard
9,0x160f,CUS_0x21b1,February,Rick Rothackerj,28,004-07-5839,Teacher,34847.84,3037.986667,2.0,4.0,6.0,1,Credit-Builder Loan,7,1,7.42,2.0,Good,605.03,38.550848,26 Years and 8 Months,No,18.816215,40.39123782853101,High_spent_Large_value_payments,484.5912142650067,Good


In [41]:
# Evaluate the count of unique values in each of the columns

unique_values_per_column = []

for column in dataset.columns:
    entry = {
        "Column": column,
        "Unique Values": dataset[column].nunique()}
    unique_values_per_column.append(entry)

unique_values_per_column = pd.DataFrame(unique_values_per_column)
print(unique_values_per_column)

                      Column  Unique Values
0                         ID         100000
1                Customer_ID          12500
2                      Month             44
3                       Name          10139
4                        Age           1799
5                        SSN          12501
6                 Occupation             31
7              Annual_Income          18936
8      Monthly_Inhand_Salary          13235
9          Num_Bank_Accounts            946
10           Num_Credit_Card           1180
11             Interest_Rate           1750
12               Num_of_Loan            434
13              Type_of_Loan           6260
14       Delay_from_due_date             73
15    Num_of_Delayed_Payment            762
16      Changed_Credit_Limit           4384
17      Num_Credit_Inquiries           1223
18                Credit_Mix              4
19          Outstanding_Debt          13178
20  Credit_Utilization_Ratio         100000
21        Credit_History_Age    

In [ ]:
# Evaluate the count of unique values in each of the columns

mean_unique_values_per_customer = []

for column in dataset.columns:
    if column != "Customer_ID":
        entry = {
            "Column": column,
            "Unique Values": dataset.groupby("Customer_ID")[column].nunique().mean()}
        mean_unique_values_per_customer.append(entry)

mean_unique_values_per_customer = pd.DataFrame(mean_unique_values_per_customer)
print(mean_unique_values_per_customer)

                      Column  Unique Values
0                         ID        8.00000
1                      Month        8.00000
2                       Name        1.00000
3                        Age        2.13088
4                        SSN        1.36872
5                 Occupation        1.51464
6              Annual_Income        1.51608
7      Monthly_Inhand_Salary        1.05968
8          Num_Bank_Accounts        1.13056
9            Num_Credit_Card        1.20720
10             Interest_Rate        1.16264
11               Num_of_Loan        1.63288
12              Type_of_Loan        0.88592
13       Delay_from_due_date        2.98736
14    Num_of_Delayed_Payment        3.33824
15      Changed_Credit_Limit        2.56536
16      Num_Credit_Inquiries        1.69848
17                Credit_Mix        1.83816
18          Outstanding_Debt        1.07848
19  Credit_Utilization_Ratio        8.00000
20        Credit_History_Age        7.27760
21     Payment_of_Min_Amount    

In [ ]:
# For those values that are very near to a round number, but not quite there (1.05 instead of 1), can the problem be solved by substituting the missing values? If not, we know that the value might take more than two numbers and we would have to dig deeper for data cleaning.


mean_unique_values_per_customer = []

for column in dataset.columns:
    if column != "Customer_ID":
        # Identify customers with NO missing values in the column
        no_missing_customers = (
            dataset[["Customer_ID", column]]
            .groupby("Customer_ID")
            .filter(lambda x: x[column].isnull().sum() == 0)
        )

        # Calculate mean number of unique values per customer for this column
        entry = {
            "Column": column,
            "Unique Values": no_missing_customers.groupby("Customer_ID")[column].nunique().mean()
        }

        mean_unique_values_per_customer.append(entry)

mean_unique_values_per_customer = pd.DataFrame(mean_unique_values_per_customer)
print(mean_unique_values_per_customer)


                      Column  Unique Values
0                         ID       8.000000
1                      Month       8.000000
2                       Name       1.000000
3                        Age       2.131679
4                        SSN       1.368442
5                 Occupation       1.515029
6              Annual_Income       1.515996
7      Monthly_Inhand_Salary       1.063167
8          Num_Bank_Accounts       1.130389
9            Num_Credit_Card       1.207172
10             Interest_Rate       1.162302
11               Num_of_Loan       1.633411
12              Type_of_Loan       1.000000
13       Delay_from_due_date       2.987360
14    Num_of_Delayed_Payment       3.480171
15      Changed_Credit_Limit       2.565360
16      Num_Credit_Inquiries       1.708142
17                Credit_Mix       1.838160
18          Outstanding_Debt       1.078480
19  Credit_Utilization_Ratio       8.000000
20        Credit_History_Age       8.000000
21     Payment_of_Min_Amount    

# Things to do

- Essential to understand every variable of the dataset
- Check whether the columns are the right type
    - If no, force the correct type.
- After knowing data:
    - Plot and and see summary statistics
    - Check the unique values of each
    - Check if the data make makes sense
        - If not, see the most correct way to deal with it
- Check multicollinearity? How important is it when you would run a more advanced model?
- Check class imbalance in target variable. Is it necessary to do any sort of balancing?
- Would it be good to apply feature enginnering?
- Remove duplicates

In [ ]:
dataset.describe(include="all")

In [ ]:
# Checking the logic and information of each of the variables

# Why do we have 100k customer IDs and just 12500 customerIDs?

# dataset.groupby("Customer_ID")["ID"].count().describe() 
# With this line of code we discover that each Customer_ID has 8 corresponding IDs.


# Which information is aggregated by Customer_ID? To know this, we gorup variables by Customer_ID and evaluate whether the number of unique value is 1 or not.

variables_by_customer_id = []
variables_by_id = []
other_variables = []


for variable in dataset.drop("Customer_ID", axis = 1).columns:
    summary_stats = dataset.groupby("Customer_ID")[variable].nunique().describe()
    if summary_stats["mean"] == 1:
        variables_by_customer_id.append(variable)
    elif summary_stats["mean"] == 8:
        variables_by_id.append(variable)
    else:
        other_variables.append(variable)

print(f'''

Variables by Customer_ID: {variables_by_customer_id}
Variables by ID: {variables_by_id}
Other variables: {other_variables}

      ''')









# If each ID corresponds to an specific product and this information is real, it means that each customer really has 8 products
# What value does month bring in? Month of what?
# dataset["Month"].unique()

In [ ]:
variables_by_customer_id = []
variables_by_id = []
other_variables = []

for variable in dataset.drop("Customer_ID", axis = 1).columns:
    dataset_cleaned = dataset.dropna(subset=[variable])
    summary_stats = dataset_cleaned.groupby("Customer_ID")[variable].nunique().describe()
    top_5_frequent = dataset_cleaned[variable].value_counts().head(5)
    least_5_frequent = dataset_cleaned[variable].value_counts().tail(5)
    print(summary_stats)
    print(f'''
          Top: {top_5_frequent},
          Least: {least_5_frequent}
          ''')
    if dataset.groupby("Customer_ID")[variable].nunique().describe()["mean"] == 1:
        variables_by_customer_id.append(variable)
    elif dataset.groupby("Customer_ID")[variable].nunique().describe()["mean"] == 8:
        variables_by_id.append(variable)
    else:
        other_variables.append(variable)
    


In [ ]:
# Are there duplicated values in the dataset?

dataset.duplicated().sum()

# No duplicated rows